# Swarm Simulator for Outer Space

This swarm simulator will explore different swarm algorithms for an application in outer space. This approach will divide a plot into grid squares, which can then be analysed to determine and control the behaviour of each individual agent in the swarm.

In [1]:
#imports
import sympy as sym
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import deque
from scipy.spatial.distance import cdist, pdist, euclidean
from sympy import sin, cos, symbols, lambdify

from robot import Robot
from dust_devil import DustDevil
from live_functions_tracking import update_decay, initialise,random_position, grid_center,positions,bounce,magnitude,unit,division_check,physics_walk,dust_check,update_timestep,cluster_function,G_transition,dist,random_walk,dust,trajectory_dust,update_dust,load_positions,pre_initialise

import random
import datetime
import os
import math
import sys

import Processing_Functions_Tracking


import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import json
import pickle

from ast import literal_eval

app = dash.Dash(__name__)

In [2]:
#Plotly offline mode
init_notebook_mode(connected=True)

In [3]:
#Function to call physics based area coverage algorithm
def physics_walk_tracking(swarm,G,power,R,max_force,multiply,timestep):
    '''
    Updates the robots velocities based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
            Returns:
                   None

    '''
    #setting initial constants
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    
    #retrieving x and y positions
    x,y = positions(swarm)
    
    #looping through swarm with each individual robot
    for i in range(len(swarm)):
        
        #selecting robots one by one
        robot = swarm[i]
        #print("Robot Position: ", [robot.x,robot.y])
        
        x_copy,y_copy = x.copy(),y.copy()
        
        #initialising the forces as 0
        force = np.array([0,0])

        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))

        #print("Distance: ",distance)
        #print("Non Zero Distance:", np.nonzero(distance))
        #determining the robots within 1.5R distance
        distance_local = np.logical_and(distance>0,distance<multiply*R)
        neighbourhood = np.where(distance_local)
        list_neighbours = list(neighbourhood[0])
        #print("Indices of local: ",list_neighbours)
        #print(len(np.where(distance_local)))
        #print("Booleans Local: ",distance_local)
        #print(type(distance_local))
        #creating the x and y distance matrices for robots within the local 1.5R distance
        neighbours = [swarm[i] for i in list_neighbours]
        #print("Neighbours:",positions(neighbours))
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        
        #initialising force changes
        force_change_dir = 0
        
        #number of robots within the neighbourhood
        robot_number = len(distance_local)
        

        #looping through the current robots neighbourhood
        for j in range(len(neighbours)):
            
            neighbour = neighbours[j]
            
            #storing current position 
            current_position = np.array([position[0,j],position[1,j]])
                

            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])


            #calculating the force
            numerator = (G*(robot.mass*neighbour.mass))
            denominator = mag**2


            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)

            #calculating the unit vector of the position
            distance_unit = unit(current_position)

            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>robot.R):
                force_change_dir = distance_unit

            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<robot.R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])

            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if((force_delta[0]**2+force_delta[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force_delta[0],force_delta[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force_delta[0] = updating_force[0]
                force_delta[1] = updating_force[1]

            #calculating new force
            force = force+force_delta
            

        #calculating the change in velocity
        delta_vx = (force[0])*timestep/robot.mass
        delta_vy = (force[1])*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy

       #keeping the velocity within the maximum velocity of the robot
        if((velocity_x**2+velocity_y**2)>((robot.max_velocity)**2)):
            #calculating unit vector of the velocity
            unit_velocity = unit([velocity_x,velocity_y])
            
            #multiplying it by the maximum velocity
            velocity = unit_velocity*robot.max_velocity
            
            #setting the new velocity equal to the respective updated maximum velocity components
            velocity_x = velocity[0]
            velocity_y = velocity[1]

        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    


In [4]:

#Function to call physics based area coverage algorithm
def physics_walk_honey(swarm,G,power,R,max_force,multiply,timestep,lattice,velocity_coefficient):
    '''
    Updates the robot position based on a physicomimetics algorithm

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None

    '''


    #print("Physics Called \n===============")
    #setting initial constants
    #setting initial constants
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    x,y = np.array(positions(swarm))
    type_0,type_1 = split(swarm)
    x_0,y_0 = np.array(positions(type_0))
    x_1,y_1 = np.array(positions(type_1))
    #print("Length 0:",len(x_0))
    #print("Length 1: ",len(x_1))
    
    #looping through each robot in the swarm
    for i in range(len(swarm)):
        
        #selecting robots one by one
        robot = swarm[i]
        
        #initialising the forces as 0
        force = np.array([0,0])
        
        R_0 = R
        R_1 = R
        #x_dist_unsorted,y_dist_unsorted = dist(robot,swarm)
        
        #finding the distance between the current robot and the other robots
        delta_x0,delta_y0 = dist(robot,x_0,y_0)
        delta_x1,delta_y1 = dist(robot,x_1,y_1)
        """#finding the distance between the current robot and the other robots
        delta_x0,delta_y0 = dist(robot,x,y)
        delta_x1,delta_y1 = dist(robot,x,y)"""
        
        #calculating the distance from the robots to the current robot, split up according to robot type
        distance_0 = np.sqrt(np.square(x_0)+np.square(y_0))
        distance_1 = np.sqrt(np.square(x_1)+np.square(y_1))
        
        #finding the corresponding booleans
        multiply_like = 1.3
        multiply_unlike = 1.7
        constant = 0
        if(robot.identifier == 0):
            R_0 = R
            R_1 = R
            distance_local_0 = np.logical_and(distance_0>0,distance_0<multiply_like*R)
            distance_local_1 = np.logical_and(distance_1>0,distance_1<multiply_unlike*R)
            constant_0 = 1/lattice
            constant_1 = 1
            neighbourhood_0 = np.where(distance_local_0)
            list_neighbours_0 = list(neighbourhood_0[0])
            neighbours_0 = [type_0[i] for i in list_neighbours_0]
            
            neighbourhood_1 = np.where(distance_local_1)
            list_neighbours_1 = list(neighbourhood_1[0])
            neighbours_1 = [type_1[i] for i in list_neighbours_1]
            
        if(robot.identifier == 1):
            R_0 = R
            R_1 = R
            distance_local_0 = np.logical_and(distance_0>0,distance_0<multiply_unlike*R)
            distance_local_1 = np.logical_and(distance_1>0,distance_1<multiply_like*R)
            constant_0 = 1
            constant_1 = 1/lattice
            neighbourhood_0 = np.where(distance_local_0)
            list_neighbours_0 = list(neighbourhood_0[0])
            neighbours_0 = [type_0[i] for i in list_neighbours_0]
            
            neighbourhood_1 = np.where(distance_local_1)
            list_neighbours_1 = list(neighbourhood_1[0])
            neighbours_1 = [type_1[i] for i in list_neighbours_1]
            
        
        x_dist_0 = delta_x0[distance_local_0]
        y_dist_0 = delta_y0[distance_local_0]    
        positions_0 = np.array([x_dist_0,y_dist_0])
        
        x_dist_1 = delta_x1[distance_local_1]
        y_dist_1 = delta_y1[distance_local_1]
        positions_1 = np.array([x_dist_1,y_dist_1])
        
        force_0 = artificial_gravity(swarm,positions_0,robot,max_force,R_0,G,constant_0)
        force_1 = artificial_gravity(swarm,positions_1,robot,max_force,R_1,G,constant_1)
        
        friction = random.uniform(-0.1,0.1)
        force = force_0 + force_1 #+ friction

        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = velocity_coefficient*(robot.x_velocity + delta_vx)
        velocity_y = velocity_coefficient*(robot.y_velocity + delta_vy)
        
    
        #keeping the velocity within the maximum velocity of the robot
        if((velocity_x**2+velocity_y**2)>((robot.max_velocity)**2)):
            #calculating unit vector of the velocity
            unit_velocity = unit([velocity_x,velocity_y])
            
            #multiplying it by the maximum velocity
            velocity = unit_velocity*robot.max_velocity
            
            #setting the new velocity equal to the respective updated maximum velocity components
            velocity_x = velocity[0]
            velocity_y = velocity[1]
        
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)



In [5]:
#Function to calculate artificial gravity forces
def artificial_gravity(swarm,neighbours, robot, max_force,R,G,constant):
        print("R: ",R)
        print("Constant: ", constant)
        force = np.array([0,0])
        force_change_dir = 0
        #print(neighbours)
        for x,y in zip(neighbours[0],neighbours[1]):
            #neighbour = neighbours[i]
            #print(i)
            #storing current position 
            current_position = np.array([x,y])

            
            #calculating magnitude of positions
            mag_unshifted = (magnitude(current_position[0],current_position[1]))
            #print(mag_unshifted)
            if(mag_unshifted > R):
                print("Larger Distance Value: ", mag_unshifted)
                if(mag_unshifted>1.7*R):
                    print("Error! Distance is too large. Value is ",mag_unshifted)
            mag = mag_unshifted*constant
            

            
            #calculating the force
            numerator = (G*(robot.mass**2))
            denominator = mag**2

            
            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)
            
            #calculating the unit vector of the position
            distance_unit = unit(current_position)
            
            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>R):
                force_change_dir = distance_unit
            
            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])
       
            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if((force_delta[0]**2+force_delta[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force_delta[0],force_delta[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force_delta[0] = updating_force[0]
                force_delta[1] = updating_force[1]
            force = force+force_delta
            
        #setting force maximum for sum of force contributions
        """if((force[0]**2+force[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force[0],force[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force[0] = updating_force[0]
                force[1] = updating_force[1]"""
        return force
    

In [6]:
#Function to calculate distance between a robot and the swarm
def dist(robot, x,y):
    #finding the distance between the current robot and the other robots
    x_dist = x-robot.x
    y_dist = y-robot.y
    return x_dist,y_dist

In [7]:
#Function to split swarm into different types
def split(swarm):
    type_0 = []
    type_1 = []
    for robot in swarm:
        if(robot.identifier == 0):
            type_0.append(robot)
        if(robot.identifier == 1):
            type_1.append(robot)

            
    return type_0,type_1
    

In [8]:
def broadcast(swarm, detection_list, set_R):
    #retrieving x and y positions
    x,y = positions(swarm)
   
    detecting = np.where(detection_list)
    list_detected = list(detecting[0])
    for i in list_detected:
        robot = swarm[i]
        x_copy,y_copy = x.copy(),y.copy()
        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))
       
        #determining the robots within 1.5R distance
        distance_local = np.logical_and(distance>0,distance<multiply*R)   
        
        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        
        neighbourhood = np.where(distance_local)
        list_neighbours = list(neighbourhood[0])

        neighbours = [swarm[i] for i in list_neighbours]
        #looping through the current robots neighbourhood
        for j in list_neighbours:
            swarm[j].R = set_R
    
            

In [9]:
#Function to calculate the G transition parameter
def G_square(max_force,R,power):
    '''
    Calculates the transition value between liquid and solid states for the G parameter

            Parameters:
                   max_force (float): the maximum float limit on the force
                   R (float): the chosen R seperation float value
                   power (float): the chosen power for the artificial gravity equation
                   
            Returns:
                   G_transition (float): the G transition value for the current swarm setup

    '''
    G_transition = (max_force*(R**power))/(2+2*math.sqrt(2))
    return G_transition

## Physics Constants

In [10]:
#setting physics constants
power = 2
R = 20
#max_force = 1
multiply = 1.5

In [11]:
#initialising parameters
conversion_ms = 1/3.6 #km/h to m/s conversion actor
timer = 0


#Robot Parameters
robot_number = 100
robot_speed = 2 #m/s
robot_radius = 1 
robot_mass = 1 #kgfro
x_robot = []
y_robot = []
min_neighbour = []
gen_random = 0 #the random angle

#dust devil parameters
dust_time = 170 #time in seconds the dust devil will survive 
dust_speed = 5 #m/s
dust_radius = 6
dust_devils = []
detection = 100

#Area Parameters
area = 2 #area in km^2 
side = 1 #each side length in m
per_square_km = 100/(60*60*24)
probability_dust = per_square_km*4#(per_square_km*area) #number of dust devils per second






In [12]:
lattice_options = [math.sqrt(3),math.sqrt(2)]
G_scale = [0.9,0.95,1,1.05,1.1]
initialise_array = [20]
lattice = lattice_options[0]
robot = 100#100
time = 2
robot_speed_array = [1,2,3,4,5,6,7,8,9,10]
max_force_array = [1]
velocity_coefficient_array = [0.5]
original_path = "../Experiments/Temp/"
try:
    os.mkdir(original_path)
except OSError:
    print ("Failure: Directory creation of %s failed" % original_path)
else:
    print ("Success: Directory creation of %s succeeded" % original_path)

values = []
timestep = 1
frequency = 1
for h in range(len(velocity_coefficient_array)):
    velocity_coefficient = velocity_coefficient_array[h]
    code_velocity = "No_G_Scaling"#_" + str(velocity_coefficient)
    outer_path = (original_path + code_velocity + "/")
    try:
        os.mkdir(outer_path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % outer_path)
    else:
        print ("Success: Directory creation of %s succeeded" % outer_path)

    for i in range(len(max_force_array)):
        max_force = max_force_array[i]
        G_state = G_transition(max_force,R,power)
        G_array = [G_state]#(np.arange(G_state*0.8, G_state*1.2, 0.02*G_state))

        for G_parameter in G_array:
            
            random_seed = b'}\xf4[\xcc\xd0\x96m\x1c'
            random.seed(random_seed)

            performance_metric = []
            code = "Max_Force_" + str(max_force)
            path = (outer_path + code + "/")
            try:
                os.mkdir(path)
            except OSError:
                print ("Failure: Directory creation of %s failed" % path)
            else:
                print ("Success: Directory creation of %s succeeded" % path)


            image = (path + "Images/")
            try:
                os.mkdir(image)
            except OSError:
                print ("Failure: Directory creation of %s failed" % image)
            else:
                print ("Success: Directory creation of %s succeeded" % image)
           
            #for initial in initial_array:

            #only set seed if eliminating bias
            #random_seed = os.urandom(8)
            #random.seed(random_seed)
            #setting up the robot setup
            swarm = initialise(robot, side, initialise_array[0],robot_speed,R)

            store_robots = np.zeros((2,robot,(time*timestep)))
            store_dust = np.zeros((2,robot,(time*timestep)))
            min_neighbours = []
            cluster_average = []
            store_dust = []
            dust_devils = []
            measurement_events = 0
            detection_metric = 0
            dust_count = 0 
            total_detection = []
            total_collision = []
            total_dust = []
            print(G_parameter)
            code = "G_Parameter_" + str(G_parameter)
            path = (path + code + "/")
            try:
                os.mkdir(path)
            except OSError:
                print ("Failure: Directory creation of %s failed" % path)
            else:
                print ("Success: Directory creation of %s succeeded" % path)
            for j in range(0,time*frequency):
                print(j)
                update_decay(swarm,R)
                #updating robots and storing robot/dust devil positions
                store_robots[:,:,j] = positions(swarm)
                store_dust.append(positions(dust_devils))

                #checking if 1 second has passed and updating velocities of the robots via the physics walk algorithm and updating status of dust devils
                if(j%frequency==0):
                #    dust_count = dust_count + dust(dust_devils,1,side,j,dust_speed,dust_time,timestep,frequency)
                   physics_walk_honey(swarm,G_parameter,power,R,max_force,multiply,1,lattice,velocity_coefficient)    

                #calling update timestep method which updates positions of robots, and returns performance metrics
                measurement_events_change,detection_metric_change = update_timestep(swarm,dust_devils,timestep,frequency,min_neighbours,cluster_average,detection,R,multiply,20,5)
                measurement_events = measurement_events + measurement_events_change
                detection_metric = detection_metric + detection_metric_change

                #appending performance metrics into list for storage/post processing
                total_detection.append(detection_metric)
                total_collision.append(measurement_events)

                #updates dust devils positions
                update_dust(dust_devils)

                #storing the number of dust devils generated
                total_dust.append(dust_count)

            #saving robot positions
            np.save(path + "Robots.npy",store_robots)

            #saving dust devil positions
            with open( path + "dust.txt", "w") as f:
                json.dump(store_dust, f)

            #saving performance metrics - these could not be stored in excel due to storage limits
            np.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))
            np.save(path + "Cluster Average.npy", np.array(cluster_average))
            np.save(path + "Measurement Events Count.npy", np.array(total_collision))
            np.save(path + "Number of Dust Devils Detected.npy", np.array(total_detection))
            np.save(path + "Number of Dust Devils Generated.npy", np.array(total_dust))


            #adding values tested for a values excel spreadsheet for easy viewing
            values.append([len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,min_neighbours[time*frequency-1],cluster_average[time*frequency-1],random_seed, G_state,measurement_events,detection_metric,dust_count])

            #the titles of the variables in the values array
            values_title = ['Number of Robots',
                                    'Timestep',
                                    'G', 
                                    'Power', 
                                    'Communication Range', 
                                    'Multiplier',
                                    'Max Force',
                                    'Max Speed',
                                    'Minimum Distance to Neighbours',
                                    'Average Cluster Size',
                                    'Seed',
                                    'G Transition',
                                    'Measurement Events Count',
                                    'Number of Dust Devils Detected',
                                    'Total Number of Dust Devils',
                ]

            #storing the constants used in the simulation
            data = {'Values':[len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,random_seed, G_state]}
            titles =['Number of Robots',
                                    'G',
                                    'Timestep', 
                                    'Power',
                                    'Communication Range', 
                                    'Multiplier',
                                    'Max Force',
                                    'Max Speed',
                                    'Seed',
                                    'G Transition',
                ]
            #setting the constants as a panda dataframe
            constants = pd.DataFrame(data, index = titles
                                   ) 

            constants.to_excel(path + "Constants.xlsx")

            #setting the start paths for the graphs and the tables
            graph_start_path = image + code + "- Graph_Beginning.png"
            table_start_path = image + "/" + code + "- Table_Beginning.png"

            #using the processing functions to create plotly graphs and tables for the figures in the first timestep
            graph_start = Processing_Functions_Tracking.graph_figure(store_robots,0,frequency,code)
            graph_start.write_image(graph_start_path)
            table_start = Processing_Functions_Tracking.table_figure(store_robots,0,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
            table_start.write_image(table_start_path)

            #combining the tables and the graphs using pillow
            Processing_Functions_Tracking.combine(graph_start_path,table_start_path)

            #setting the end paths for the graphs and the tables
            graph_end_path = image + code + "- Graph_End.png"
            table_end_path = image + "/" + code + "- Table_End.png"

            #using the processing functions to create plotly graphs and tables for the figures in the last timestep
            graph_end = Processing_Functions_Tracking.graph_figure(store_robots,time-1,frequency,code)
            graph_end.write_image(image + "/" + code + "- Graph_End.png")
            table_end = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
            table_end.write_image(image + "/" + code + "- Table_End.png")

            #combining the tables and the graphs using pillow
            Processing_Functions_Tracking.combine(graph_end_path,table_end_path)

            #plotting performance of the average of minimum neighbouring distance metric
            performance = Processing_Functions_Tracking.performance_graph(min_neighbours,np.linspace(0,len(min_neighbours),len(min_neighbours)*frequency,endpoint = False),frequency,code,"Time (s)","Minimum Average Neighbour Distance (m)")
            performance.write_image(image + "/" + code + "- Performance.png")

            #plotting performance of the dust devil measurement metric
            performance_intercept = Processing_Functions_Tracking.performance_graph(total_collision,np.linspace(0,len(total_collision),len(total_collision)*frequency,endpoint = False),frequency,code,"Time (s)","Count of Measurement Events")
            performance_intercept.write_image(image + "/" + code + "- Intercept Performance.png")

            #plotting performance of the cluster average of the swarm
            cluster = Processing_Functions_Tracking.performance_graph(cluster_average,np.linspace(0,len(cluster_average),len(cluster_average)*frequency,endpoint = False),frequency,code,"Time (s)","Average Cluster Size")
            cluster.write_image(image + "/" + code + "- Average Cluster Size.png")




            #setting panda dataframe for the values tested
            df = pd.DataFrame(values, columns = values_title)

            #saving at as an excel spreadsheet
            df.to_excel(path + "Results.xlsx")


            #performance_metric_np = np.array(performance_metric)

            #saving performance metrics
            #np.save(original_path + "Performance Metric " + str(timestep) + ".npy", performance_metric_np)

            #plotting performance of the different timesteps
            #performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second","Number of Dust Devils Detected")
            #performance_overall.write_image(new_path + "Overall Performance.png")

    """
        performance_metric.append([robot,total_detection[-1]])


performance_metric_np = np.array(performance_metric)
np.save(new_path + "Performance Metric " + str(i) + ".npy", performance_metric_np)
performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second ","Measurement Event Count")
performance_overall.write_image(new_path + "Overall Performance.png")"""


Failure: Directory creation of ../Experiments/Temp/ failed
Failure: Directory creation of ../Experiments/Temp/No_G_Scaling/ failed
Failure: Directory creation of ../Experiments/Temp/No_G_Scaling/Max_Force_1/ failed
Failure: Directory creation of ../Experiments/Temp/No_G_Scaling/Max_Force_1/Images/ failed
115.47005383792516
Failure: Directory creation of ../Experiments/Temp/No_G_Scaling/Max_Force_1/G_Parameter_115.47005383792516/ failed
0
R:  20
Constant:  0.5773502691896258
Larger Distance Value:  21.470910553583888
Larger Distance Value:  27.459060435491963
Larger Distance Value:  28.178005607210743
Larger Distance Value:  30.675723300355934
Larger Distance Value:  35.0
Error! Distance is too large. Value is  35.0
Larger Distance Value:  39.824615503479755
Error! Distance is too large. Value is  39.824615503479755
Larger Distance Value:  26.019223662515376
Larger Distance Value:  35.22782990761707
Error! Distance is too large. Value is  35.22782990761707
Larger Distance Value:  21.587

Constant:  0.5773502691896258
Larger Distance Value:  20.396078054371138
Larger Distance Value:  20.12461179749811
Larger Distance Value:  29.68164415931166
Larger Distance Value:  31.400636936215164
Larger Distance Value:  25.298221281347036
Larger Distance Value:  20.8806130178211
Larger Distance Value:  20.615528128088304
Larger Distance Value:  20.248456731316587
Larger Distance Value:  26.92582403567252
Larger Distance Value:  29.068883707497267
Larger Distance Value:  23.706539182259394
Larger Distance Value:  23.53720459187964
Larger Distance Value:  30.01666203960727
R:  20
Constant:  0.5773502691896258
Larger Distance Value:  35.0
Error! Distance is too large. Value is  35.0
Larger Distance Value:  25.709920264364882
Larger Distance Value:  33.24154027718932
Larger Distance Value:  33.0
Larger Distance Value:  38.3275357934736
Error! Distance is too large. Value is  38.3275357934736
Larger Distance Value:  25.317977802344327
Larger Distance Value:  20.591260281974
Larger Dista

Larger Distance Value:  27.0
Larger Distance Value:  24.08318915758459
Larger Distance Value:  32.64965543462902
Larger Distance Value:  39.59797974644666
Error! Distance is too large. Value is  39.59797974644666
Larger Distance Value:  34.058772731852805
Error! Distance is too large. Value is  34.058772731852805
Larger Distance Value:  25.495097567963924
Larger Distance Value:  22.02271554554524
Larger Distance Value:  24.758836806279895
Larger Distance Value:  33.301651610693426
Larger Distance Value:  20.615528128088304
Larger Distance Value:  26.076809620810597
Larger Distance Value:  25.0
Larger Distance Value:  31.575306807693888
Larger Distance Value:  36.87817782917155
Error! Distance is too large. Value is  36.87817782917155
Larger Distance Value:  20.808652046684813
Larger Distance Value:  28.178005607210743
Larger Distance Value:  21.095023109728988
Larger Distance Value:  32.449961479175904
Larger Distance Value:  20.591260281974
R:  20
Constant:  0.5773502691896258
Larger 

Larger Distance Value:  30.886890422961002
Larger Distance Value:  31.016124838541646
Larger Distance Value:  25.238858928247925
Larger Distance Value:  21.93171219946131
Larger Distance Value:  28.442925306655784
Larger Distance Value:  20.024984394500787
R:  20
Constant:  1
Larger Distance Value:  20.591260281974
Larger Distance Value:  25.079872407968907
Larger Distance Value:  29.017236257093817
Larger Distance Value:  20.615528128088304
Larger Distance Value:  20.615528128088304
Larger Distance Value:  22.561028345356956
Larger Distance Value:  22.47220505424423
Larger Distance Value:  25.0
Larger Distance Value:  22.360679774997898
Larger Distance Value:  35.608987629529715
Error! Distance is too large. Value is  35.608987629529715
Larger Distance Value:  25.495097567963924
Larger Distance Value:  22.825424421026653
Larger Distance Value:  22.20360331117452
Larger Distance Value:  23.706539182259394
Larger Distance Value:  25.632011235952593
Larger Distance Value:  23.08679276123

R:  20
Constant:  0.5773502691896258
Larger Distance Value:  35.35533905932738
Error! Distance is too large. Value is  35.35533905932738
Larger Distance Value:  32.01562118716424
Larger Distance Value:  38.91015291668744
Error! Distance is too large. Value is  38.91015291668744
Larger Distance Value:  41.23105625617661
Error! Distance is too large. Value is  41.23105625617661
Larger Distance Value:  29.068883707497267
Larger Distance Value:  27.784887978899608
Larger Distance Value:  41.23105625617661
Error! Distance is too large. Value is  41.23105625617661
Larger Distance Value:  21.095023109728988
Larger Distance Value:  30.479501308256342
Larger Distance Value:  23.08679276123039
Larger Distance Value:  34.66987164671943
Error! Distance is too large. Value is  34.66987164671943
Larger Distance Value:  31.400636936215164
Larger Distance Value:  38.01315561749642
Error! Distance is too large. Value is  38.01315561749642
Larger Distance Value:  34.92849839314596
Error! Distance is too

...

Larger Distance Value:  25.409862872092702
R:  20
Constant:  0.5773502691896258
Larger Distance Value:  37.179100303436776
Error! Distance is too large. Value is  37.179100303436776
Larger Distance Value:  29.146888167951015
Larger Distance Value:  32.646596309093354
Larger Distance Value:  35.00756684608943
Error! Distance is too large. Value is  35.00756684608943
Larger Distance Value:  41.555133471518026
Error! Distance is too large. Value is  41.555133471518026
Larger Distance Value:  20.6418353898531
Larger Distance Value:  30.343244686053463
Larger Distance Value:  25.990785215635675
Larger Distance Value:  40.95336987442023
Error! Distance is too large. Value is  40.95336987442023
Larger Distance Value:  31.82976959224414
Larger Distance Value:  33.738016300785844
Larger Distance Value:  26.050794621764297
Larger Distance Value:  41.21654259000966
Error! Distance is too large. Value is  41.21654259000966
Larger Distance Value:  39.08306188253939
Error! Distance is too large. Val

Error! Distance is too large. Value is  36.41798916532468
Larger Distance Value:  38.641566277234965
Error! Distance is too large. Value is  38.641566277234965
Larger Distance Value:  30.931043867897976
Larger Distance Value:  45.77560007435354
Error! Distance is too large. Value is  45.77560007435354
Larger Distance Value:  43.3200864690725
Error! Distance is too large. Value is  43.3200864690725
Larger Distance Value:  26.691928533672538
Larger Distance Value:  33.36247524576184
Larger Distance Value:  29.084152316814773
Larger Distance Value:  26.06947137326347
Larger Distance Value:  44.09632162132302
Error! Distance is too large. Value is  44.09632162132302
Larger Distance Value:  49.904779255087355
Error! Distance is too large. Value is  49.904779255087355
Larger Distance Value:  46.1216202131537
Error! Distance is too large. Value is  46.1216202131537
Larger Distance Value:  26.4215295645243
Larger Distance Value:  29.349403175789075
Larger Distance Value:  21.32766536793714
Lar

Constant:  1
Larger Distance Value:  28.24712363148303
Larger Distance Value:  29.909216995628825
Larger Distance Value:  42.59407073388273
Error! Distance is too large. Value is  42.59407073388273
Larger Distance Value:  35.347615547951634
Error! Distance is too large. Value is  35.347615547951634
Larger Distance Value:  39.964979478580226
Error! Distance is too large. Value is  39.964979478580226
Larger Distance Value:  48.146353251181154
Error! Distance is too large. Value is  48.146353251181154
Larger Distance Value:  41.581710085616606
Error! Distance is too large. Value is  41.581710085616606
Larger Distance Value:  37.39849100062137
Error! Distance is too large. Value is  37.39849100062137
Larger Distance Value:  39.23991610502452
Error! Distance is too large. Value is  39.23991610502452
Larger Distance Value:  49.61983232978777
Error! Distance is too large. Value is  49.61983232978777
Larger Distance Value:  32.717499534065844
Larger Distance Value:  47.571147868808296
Error! D

R:  20
Constant:  0.5773502691896258
Larger Distance Value:  29.42994280099145
Larger Distance Value:  25.597674626514536
Larger Distance Value:  20.507322825444657
Larger Distance Value:  36.23510053372498
Error! Distance is too large. Value is  36.23510053372498
Larger Distance Value:  36.27333987589164
Error! Distance is too large. Value is  36.27333987589164
Larger Distance Value:  36.26747480197146
Error! Distance is too large. Value is  36.26747480197146
Larger Distance Value:  27.247601914347026
Larger Distance Value:  35.14037829799116
Error! Distance is too large. Value is  35.14037829799116
Larger Distance Value:  23.87610453618475
Larger Distance Value:  30.869993151899184
Larger Distance Value:  45.215576954920394
Error! Distance is too large. Value is  45.215576954920394
Larger Distance Value:  46.28503419346584
Error! Distance is too large. Value is  46.28503419346584
Larger Distance Value:  37.977177670215944
Error! Distance is too large. Value is  37.977177670215944
Lar

G_Parameter_115.47005383792516
G_Parameter_115.47005383792516
G_Parameter_115.47005383792516


In [13]:
"""original_path = "../Experiments/Initial_Types_Test/"
try:
    os.mkdir(original_path)
except OSError:
    print ("Failure: Directory creation of %s failed" % original_path)
else:
    print ("Success: Directory creation of %s succeeded" % original_path)
    
code = "Run_" + str(initialise_array[i])
path = (original_path + code + "/")
try:
        os.mkdir(path)
except OSError:
    print ("Failure: Directory creation of %s failed" % path)
else:
    print ("Success: Directory creation of %s succeeded" % path)


image = (original_path + "Images/")
try:
    os.mkdir(image)
except OSError:
    print ("Failure: Directory creation of %s failed" % image)
else:
    print ("Success: Directory creation of %s succeeded" % image)

#saving performance metrics - these could not be stored in excel due to storage limits
np.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))
np.save(path + "Cluster Average.npy", np.array(cluster_average))
np.save(path + "Measurement Events Count.npy", np.array(total_collision))
np.save(path + "Number of Dust Devils Detected.npy", np.array(total_detection))
np.save(path + "Number of Dust Devils Generated.npy", np.array(total_dust))


#adding values tested for a values excel spreadsheet for easy viewing
values.append([len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,min_neighbours[time*frequency-1],cluster_average[time*frequency-1],random_seed, G_state,measurement_events,detection_metric,dust_count])

#the titles of the variables in the values array
values_title = ['Number of Robots',
                        'Timestep',
                        'G', 
                        'Power', 
                        'Communication Range', 
                        'Multiplier',
                        'Max Force',
                        'Max Speed',
                        'Minimum Distance to Neighbours',
                        'Average Cluster Size',
                        'Seed',
                        'G Transition',
                        'Measurement Events Count',
                        'Number of Dust Devils Detected',
                        'Total Number of Dust Devils',
    ]

#storing the constants used in the simulation
data = {'Values':[len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,random_seed, G_state]}
titles =['Number of Robots',
                        'G',
                        'Timestep', 
                        'Power',
                        'Communication Range', 
                        'Multiplier',
                        'Max Force',
                        'Max Speed',
                        'Seed',
                        'G Transition',
    ]
#setting the constants as a panda dataframe
constants = pd.DataFrame(data, index = titles
                       ) 

constants.to_excel(path + "Constants.xlsx")

#setting the start paths for the graphs and the tables
graph_start_path = image + code + "- Graph_Beginning.png"
table_start_path = image + "/" + code + "- Table_Beginning.png"

#using the processing functions to create plotly graphs and tables for the figures in the first timestep
graph_start = Processing_Functions_Tracking.graph_figure(store_robots,0,frequency,code)
graph_start.write_image(graph_start_path)
table_start = Processing_Functions_Tracking.table_figure(store_robots,0,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
table_start.write_image(table_start_path)

#combining the tables and the graphs using pillow
Processing_Functions_Tracking.combine(graph_start_path,table_start_path)

#setting the end paths for the graphs and the tables
graph_end_path = image + code + "- Graph_End.png"
table_end_path = image + "/" + code + "- Table_End.png"

#using the processing functions to create plotly graphs and tables for the figures in the last timestep
graph_end = Processing_Functions_Tracking.graph_figure(store_robots,time-1,frequency,code)
graph_end.write_image(image + "/" + code + "- Graph_End.png")
table_end = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
table_end.write_image(image + "/" + code + "- Table_End.png")

#combining the tables and the graphs using pillow
Processing_Functions_Tracking.combine(graph_end_path,table_end_path)

#plotting performance of the average of minimum neighbouring distance metric
performance = Processing_Functions_Tracking.performance_graph(min_neighbours,np.linspace(0,len(min_neighbours),len(min_neighbours)*frequency,endpoint = False),frequency,code,"Time (s)","Minimum Average Neighbour Distance (m)")
performance.write_image(image + "/" + code + "- Performance.png")

#plotting performance of the dust devil measurement metric
performance_intercept = Processing_Functions_Tracking.performance_graph(total_collision,np.linspace(0,len(total_collision),len(total_collision)*frequency,endpoint = False),frequency,code,"Time (s)","Count of Measurement Events")
performance_intercept.write_image(image + "/" + code + "- Intercept Performance.png")

#plotting performance of the cluster average of the swarm
cluster = Processing_Functions_Tracking.performance_graph(cluster_average,np.linspace(0,len(cluster_average),len(cluster_average)*frequency,endpoint = False),frequency,code,"Time (s)","Average Cluster Size")
cluster.write_image(image + "/" + code + "- Average Cluster Size.png")




#setting panda dataframe for the values tested
df = pd.DataFrame(values, columns = values_title)

#saving at as an excel spreadsheet
df.to_excel(path + "Results.xlsx")


#performance_metric_np = np.array(performance_metric)

#saving performance metrics
#np.save(original_path + "Performance Metric " + str(timestep) + ".npy", performance_metric_np)

#plotting performance of the different timesteps
#performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second","Number of Dust Devils Detected")
#performance_overall.write_image(new_path + "Overall Performance.png")
"""

'original_path = "../Experiments/Initial_Types_Test/"\ntry:\n    os.mkdir(original_path)\nexcept OSError:\n    print ("Failure: Directory creation of %s failed" % original_path)\nelse:\n    print ("Success: Directory creation of %s succeeded" % original_path)\n    \ncode = "Run_" + str(initialise_array[i])\npath = (original_path + code + "/")\ntry:\n        os.mkdir(path)\nexcept OSError:\n    print ("Failure: Directory creation of %s failed" % path)\nelse:\n    print ("Success: Directory creation of %s succeeded" % path)\n\n\nimage = (original_path + "Images/")\ntry:\n    os.mkdir(image)\nexcept OSError:\n    print ("Failure: Directory creation of %s failed" % image)\nelse:\n    print ("Success: Directory creation of %s succeeded" % image)\n\n#saving performance metrics - these could not be stored in excel due to storage limits\nnp.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))\nnp.save(path + "Cluster Average.npy", np.array(cluster_average))\nnp.save(path 

In [14]:
path =  "Experiments/Types_Testing/Run_1/"
store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
     dust = json.load(f)

constants = pd.read_excel(path + "Constants.xlsx", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'Experiments/Types_Testing/Run_1/Robots.npy'

In [ ]:
#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [ ]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    

    position_robot = store_robots[:,:,timer]
    position_dust = store_dust[timer]
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-1000, 1000]),xaxis = dict(range=[-1000,1000]))
        }
    )

    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(255, 182, 193, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment </b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 1
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig = Processing_Functions_Tracking.table_figure(store_robots,timer,timestep,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
                
    fig.update_layout(width = 650, height = 800,margin=dict(
    t=200, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.2')

In [ ]:
def R_return(swarm):
    R_array = []
    for i in swarm:
        R_array.append(i.R)
    return R_array

In [ ]:
print(R_return(swarm))
print(positions(swarm))